In [1]:
# Import libraries
import pandas as pd
from bokeh.palettes import cividis
from bokeh.models import CrosshairTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# Load the election data
cols = ['year', 'state', 'candidate', 'party_detailed', 'candidatevotes', 'totalvotes']
elections_data = pd.read_csv('1976-2020-president.csv', usecols=cols, dtype={'year': str})
party = elections_data['party_detailed']
dem_rep_data = elections_data[(party == 'DEMOCRAT') | (party == 'REPUBLICAN')]

# Group data by state
state_groups = dem_rep_data.groupby('state')
years = elections_data['year'].unique()

#Palette for the plot
palette = cividis(len(state_groups))

# Figure
p = figure(tooltips='$name @x: @y%', title='Democrat/Republican percentage of total vote by state', x_range=years)

state_lines = {}

# For each group in the state_groups
for i, group in enumerate(state_groups.groups):
    state_data = state_groups.get_group(group)
    party_data = state_data[state_data['party_detailed'] == 'DEMOCRAT']
    percentages = party_data['candidatevotes'] / party_data['totalvotes'] * 100

    line = p.line(x=party_data['year'], y=percentages, color=palette[i], name=group)
    state_lines[group] = line

#Crosshair tool to the plot
p.add_tools(CrosshairTool())

# Display the plot
show(p)

Loading BokehJS ...

FileNotFoundError: [Errno 2] No such file or directory: '1976-2020-president.csv'

In [ ]:
from ipywidgets import interact, SelectMultiple, IntRangeSlider, Dropdown
from bokeh.io import push_notebook

handle = show(p, notebook_handle=True)

# Multiple state selector
states_select = SelectMultiple(options=state_groups.groups.keys(), rows=10, description='Show states (use ctrl or shift to select multiple):',)
@interact(states=states_select)
def update(states):
    for state_line_name, state_line in state_lines.items():
        if state_line_name in states:
            state_line.visible = True
        else:
            state_line.visible = False

    push_notebook(handle=handle)

min_year = int(years[0])
max_year = int(years[-1])

# Year range slider
year_range = IntRangeSlider(value=[min_year, max_year], min=min_year, max=max_year, step = 4)

@interact(years=year_range)
def update(years):
    x_years = range(years[0], years[1] + 1, 4)
    p.x_range.factors = list(map(str, x_years))
    push_notebook(handle=handle)

# Party dropdown
party_select = Dropdown(options=['DEMOCRAT', 'REPUBLICAN'], value='DEMOCRAT', description='Party:',)

@interact(party=party_select)
def update(party):
    for state_line_name, state_line in state_lines.items():
        state_data = state_groups.get_group(state_line_name)
        party_data = state_data[state_data['party_detailed'] == party]
        if party_data.empty:
            state_line.visible = False
        else:
            state_line.visible = True
    push_notebook(handle=handle)


interactive(children=(SelectMultiple(description='Show states (use ctrl or shift to select multiple):', option…

interactive(children=(IntRangeSlider(value=(1976, 2020), description='years', max=2020, min=1976, step=4), Out…

interactive(children=(Dropdown(description='Party:', options=('DEMOCRAT', 'REPUBLICAN'), value='DEMOCRAT'), Ou…